# Creating a flask-bsed online message bank 

## Intro

In this blog post, we would be creating a flask-based online message bank deployed as a python web app using the google cloud service and mySQL database.

A web application is a computer program that utilizes web browsers and web technology to perform tasks over the Internet, accessible by users with internet connections (Stackpath). 

The skills you’ll need are:

- Flask fundamentals, including render_template(), the basic anatomy of templates, and user interactions.
- Database skills, including adding items to databases and displaying them.
- Basic CSS (Cascading styling sheets) in order to add a bit of personal flare to your webapp. 
- There would be a section that briefly goes over how to deploy the web app on google cloud. 

Built With
Flask - Web framework and related modules
Google Cloud - App Deployment
MySQL - Database 
Bootstrap - Styling
Jinja2 - Template formatting
wtforms - Forms



## Functionalities Overview

The app you’re going to build is a simple message bank. It should do two things:

1. Allow the user to submit messages to the bank.
2. Allow the user to view a sample of the messages currently stored in the bank.
3. Additionally, you should use CSS to make your app look attractive and interesting.


## Setting up the basics

We first create a .py file called __app.py__ where we would import modules and define the application frame. 

```python
from flask import Flask, render_template,request, redirect, url_for, g
import sqlite3

app = Flask(__name__)

# Code for different pages to be inserted below
# Begin -------
# End ---------

if __name__ == "__main__":
    app.run(port=8000, debug=True)
```

Type __python app.py__ in the terminal to run the flask app. 

Alternatively, without the app.run section, we run the following commands in the terminal:  
(Setting the environmental variables to dev and debug mode will allow you to see updates of the web app without the need to resetart the web app everytime)
```
export FLASK_ENV=development
export FLASK_DEBUG=1
flask run 
```
which will show you the following lines:

```
* Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with fsevents reloader
 * Debugger is active!
 * Debugger PIN: 331-048-817
 ```

 following the link after __Running on__ to check out your 'website'.

Next, we write the separate functions for different pages:
- __\\__: the root directory, we will have this redirected to the "home" page
- __home__: front page of the message bank
- __submit__: page for submitting the message, contains a form, allows post&get
- __view__: page for viewing random messages, contains a form, allows post&get

```python
# @app,route... are decoraters ,i.e., functions wrappers that allow you to execute
# codes before and after it that does not change the func definition
# in this case, it specifies what function would be called to render the page 
# under a specific web directory/route
@app.route("/",  methods=['GET']) 
def index():
    return redirect(url_for('home'))

@app.route("/home/",  methods=['GET', 'POST'])
def home():
    return render_template("home.html")

@app.route("/submit/", methods=['POST', 'GET'])
def submit():
    # if we access the page from another page with differnt url
    if request.method == "GET":
        return render_template("submit.html")
    # if we submit a form on this page and land on the same page again
    else: 
        # the insert_message function updates the messages database
        insert_message(request)
        # pass in the user handle to write a thank you message to the user
        return render_template("submit.html", 
                               name=request.form['name'])

@app.route("/view/",  methods=['GET', 'POST'])
def view():
    # default: gives one message
    if request.method == "GET":
        # pass in the list of messages we retrieve from the database for display
        return render_template('view.html', 
                               messages = random_messages(1))
    else: 
    # add a slider that allows users to change the num. of messages displayed
        return render_template('view.html', 
                               messages = random_messages(int(request.form['n'])))
```

## Enabling submissions  
After defining the functions for different routes, we fill in the functions for updating the message dataset based on user input and the function for displaying random messages. 



```python
# this function is used to retrive the message database 
def get_message_db():
    # g is an object provided by Flask. 
    # It is a global namespace for holding any data you want during a single app context.
    # in this case, we create a message_db variable under g to store the dataset containing user messages
    try:
            # if the db already exists, just return it
            return g.message_db
    except:
            # if not, we establish the database
            g.message_db = sqlite3.connect("messages_db.sqlite")
            cursor = g.message_db.cursor()
            # we create a data table with id (primary key), handle (user name), and message (user message)
            # called messages if it does not exist in messages_db yet
            cmd = """
            CREATE TABLE IF NOT EXISTS messages(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                handle TEXT NOT NULL,
                message TEXT NOT NULL
            );
            """
            cursor.execute(cmd)
            g.message_db.commit()
            return g.message_db
```

__Note:__ when working directly with SQL commands, it is necessary to run db.commit() after inserting a row into db in order to ensure that your row insertion has been saved.  

You should ensure that the ID number of each message is unique. One way to do this is by setting the ID number of a message equal to one plus the current number of rows in message_db.  Another way to do this is setting it as an autoincremented primary key.

Don’t forget to close the database connection within the function!


We then write the function to update the database.   
```python
# this function inserts user input message to the message_db database
def insert_message(request):
    # establish connection to the database
    db = get_message_db()
    # extract message and handle from the request form
    message = request.form['msg']
    handle = request.form['name']
    # update the database
    cmd = 'INSERT INTO messages (handle, message) VALUES (?, ?)'
    db.execute(cmd,(handle, message))
    # commit changes and close connection
    db.commit()
    db.close()
```

## Viewing Random Submissions 

In this part, we will write a function called random_messages(n) which will return a collection of n random messages from the message_db, or fewer if necessary. 

```python
# fetch n random messages from the database
# if n > # of current rows, n = # of current rows
def random_messages(n):
    # establish connection
    db = get_message_db()
    cursor = g.message_db.cursor()
    # select random messages, fetch both the handle and message
    cmd = 'SELECT handle, message FROM messages ORDER BY RANDOM() LIMIT ?'
    msgs = cursor.execute(cmd,(n,)).fetchall()
    # close connection
    db.close()
    return msgs
```

## Defining html templates

In this part, we will define the templates for all the pages: home page (home.html), message bank submission page (submit.html), and message bank random viewing page (view.html).

We will first define a base template which styles the pages we would be showing for the web apps, then have the __submit.html__ extends the __base.html__. Both htmls were to be put under a folder called __template__.
Here are more examples about extends from [flask](https://flask.palletsprojects.com/en/2.2.x/tutorial/templates/#register) and [jinja](https://jinja.palletsprojects.com/en/3.1.x/templates/#template-inheritance).

```html
<!DOCTYPE html>
<!--created by marlene lin-->

<html lang="en">
    <head>
        <!-- Required meta tags -->
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">    
        <!-- Bootstrap CSS and my own -->
        <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css" rel="stylesheet">
        <link href="/static/favicon.ico" rel="icon">
        <link href="/static/styles.css" rel="stylesheet">
         <!-- Block that allows other pages to customize title -->
        <title>Message bank: {% block title %}{% endblock %}</title>
    </head>
    <body>
        <div class="container">
            <div class="row header">
            </div>
            <div class="row">
               <div class="col-md-1 col-lg-1 col-xl-1"></div>
               <div class="col-md-3 col-lg-3 col-xl-3 sidebar">
                <!-- navbar -->
                <ul class="nav flex-column">
								<li class="nav-item">
									<p><i><a class="nav-link" href="/home"><b>Welcome: </b></a></i></p>
								</li>
								<li class="nav-item">
									<a class="nav-link" href="/submit">Write a Message</a>
								</li>
								<li class="nav-item">
									<a class="nav-link" href="/view">View messages</a>
								</li>
							</ul>
               </div>
        <!--the block where the main content is hosted -->
        <div class="col-md-8 col-lg-8 col-xl-8 maininfo">
          {% block main %}{% endblock %}
        </div>
  <div class="col-1"></div>
           </div>
           <div class="row">
               <div class="col footer">

               </div>
           </div>
       </div>
        <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/js/bootstrap.min.js" integrity="sha384-ChfqqxuZUCnJSK3+MXmPNIyE6ZbWh2IMqE241rYiqJxyMiZ6OW/JmZQ5stwEULTy" crossorigin="anonymous"></script>
    </body>
</html>
```


Next, we will define submit.html which has the following three user interface elements:

- A text box for submitting a message.
- A text box for submitting the name of the user.
- A “submit” button.  

## Additional stylings

## Message bank screenshots